In [1]:
'''
匯入套件
'''

# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

#美麗湯
from bs4 import BeautifulSoup as bs

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

#正規表達式
import re

# 下載檔案的工具
import wget

import pandas as pd


# 嘗試更換user-agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
my_options.add_argument(f"user-agent={user_agent}")

 # 加入其他可能有用的隱私設定
my_options.add_argument('--disable-blink-features=AutomationControlled')
my_options.add_argument('--disable-extensions')
my_options.add_experimental_option('excludeSwitches', ['enable-automation'])
my_options.add_experimental_option('useAutomationExtension', False)
# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options
)

In [ ]:
import requests
import json
import csv
from datetime import datetime, timezone
from time import sleep

#### 輸出的CSV
# 輸入地點
location = "象山"
output_file = f'reviews_output_{location}.csv'


#### 在Selenium中取得地點與身分驗證的1s碼(有兩種1s碼)
# 進入google map頁面
driver.get("https://www.google.com/maps?authuser=0")

# 等待搜尋框出現
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located( 
        (By.CSS_SELECTOR, 'input#searchboxinput') 
    )
)

# 取得地點的1s碼 在google map 搜尋的url中
# 尋找網頁中的搜尋框
inputElement = driver.find_element(
    By.CSS_SELECTOR, 'input#searchboxinput'
)

# 在搜尋框中輸入文字
inputElement.clear()
inputElement.send_keys(location)

# 睡個幾秒
sleep(2)

# 尋找送出按鈕
clickButton = driver.find_element(
    By.CSS_SELECTOR, 'button#searchbox-searchbutton'
)

# 送出搜尋
clickButton.click()

# 睡個幾秒 等url跳轉為包含1s碼的url
sleep(5)

# 取得地點的url取得地點的url
currentUrl = driver.current_url

# 用正則表達式轉換地點的1s碼
regex1s01 = r'0x\w+'
google1s01 = re.findall(regex1s01,currentUrl)[0] + "%3A" + re.findall(regex1s01,currentUrl)[1]

# 取得身分驗證的1s碼 在<head><script>中
html = driver.page_source
soup = bs(html, "lxml")

script_tags = soup.find_all("script")
for script in script_tags:
    if "kEI" in script.text:
        start = script.text.find("kEI='") + len("kEI='")
        end = script.text.find("'", start)
        google1s02 = script.text[start:end]
        break

#### ID不重複
unique_ids = set()

#### 計算評論數 & 有內文的評論數
total_reviews = 0
reviews_with_comments = 0

# 計數器
count = 1

# 手動取得2s 碼 (評論排序選最新 去F12 console 的 Networking 觀察評論載入時 有個listugcposts的項目 網址中2s 後方的項目為2s碼 網址其他部分貼到下方)
# 從第一頁開始抓 只要改成空字串即可 第一頁沒有2s碼
#google2s = "CAESYENBRVFDaHBDUTJrNFNVRlNTVzVEWjI5QlVEY3lSM1J1T1ZaV1psOWZSV2hCWWt4NGNFSk1NV3BXVjNZM1pYTkRUVUZCUVVGQlIyZG1YMEZ0ZEc0NVZsWm1SMEZCYVVGQg%3D%3D"
google2s = ""

#### 發出請求
# 測試用
#for i in range(2):
# 抓到沒有下一個 2s 碼為止
while True:
     
    # 手動取得網址 方法寫在上方
    url = f'https://www.google.com/maps/rpc/listugcposts?authuser=0&hl=zh-TW&gl=tw&pb=!1m6!1s{google1s01}!6m4!4m1!1e1!4m1!1e3!2m2!1i10!2s{google2s}!5m2!1s{google1s02}!7e81!8m9!2b1!3b1!5b1!7b1!12m4!1b1!2b1!4m1!1e1!11m0!13m1!1e2'
    response = requests.get(url)
    response.raise_for_status()

    raw_content = response.text.strip()

    #### 移除開頭的)]}'
    if raw_content.startswith(")]}'"):
        raw_content = raw_content[4:]

    data = json.loads(raw_content)
    
    # 沒有下一個 2s 碼則跳出迴圈
    try:
        google2s = data[1].replace('=', '%3D')
    except:
        break

    sleep(1)
    #### 提取要的資料
    with open(output_file, mode='a', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        for i in range(10):
            try:                        
                user = [2][i][0][1][4][5][0]
                user_id = [2][i][0][1][4][5][3]
                user_page = [2][i][0][1][4][2][0]
                review_id =  str(data[2][i][0][0])
                rating = str(data[2][i][0][2][0][0])
                timestamp_seconds = data[2][i][0][1][2] // 1000000
                formatted_date = datetime.fromtimestamp(timestamp_seconds, tz=timezone.utc).strftime('%Y-%m-%d')
                
                try:
                    comment = [2][i][0][2][15][0][0]
                except:
                    comment = ""

                try:                   
                    language = [2][i][0][2][14][0]
                except:
                    language = ""

                try:
                    translated_comment = [2][i][0][2][15][1][0]
                except:
                    translated_comment = ""
                # 確保 ID 唯一，不重複儲存
                if review_id not in unique_ids:
                    writer.writerow([review_id, rating, formatted_date, comment, language, translated_comment])
                    unique_ids.add(review_id)
                    total_reviews += 1  # 增加計數
                    if comment.strip():  # 若評論內容不為空
                            reviews_with_comments += 1
            except Exception as e:
                print(f"評論 {i + 1}: 無評論或格式錯誤 ({e})")
    print(f"現在在第 {count} 頁")
    print(f"總共存入 {total_reviews} 筆評論")
    print(f"其中有 {reviews_with_comments} 筆評論有內文")
    count += 1
print("沒有下一頁")

In [ ]:
# 若意外中斷可以取得該頁 2s 碼接續
print(google2s)